In [8]:
from numpy.random import gamma, exponential
from numpy import log
from random import random
import pandas as pd

In [9]:
def exp(l, u):# экспоненциальное распределение (для определения времени поступления детали)
    return -l*log(u)


def type_detail(u):# тип создаваемой детали
    if u <= 0.3:
        return 1
    elif u > 0.3 and u <= 0.5:
        return 2
    else:
        return 3


def min_time_index_last(history, type_d):
    time_min = 8 * 60
    for item in history:
        if item[0] == type_d and not item[3]:
            if time_min > item[2]:
                time_min = item[2]
                id_plat = item[4]
    return time_min, id_plat

In [10]:
# станки на платформах. id_платформы: {id_станка : блокирован/готов к работе}
platform = {
    "1" : {
        "1" : True,
        "2" : True,
        "3" : True,
        "4" : True,
        "5" : True
    },
    "2" : {
        "1" : True,
        "2" : True,
        "3" : True,
        "4" : True,
        "5" : True
    },
    "3" : {
        "1" : True,
        "2" : True,
        "3" : True,
        "4" : True,
        "5" : True
    }
}

# порядок выполнения работ
work = {
    "1" : [1, 1, 1, 1],
    "2" : [2, 2, 2],
    "3" : [3, 3, 3, 3, 3]
}

# время выполнения каждой работы
time_work = {
    "1" : [.25, .15, .1, .3],
    "2" : [.15, .2, .3],
    "3" : [.15, .1, .35, .2, .2]
}

In [11]:
gamma_form = 2 # форма гамма распредления
l = 1/15 # 1/15 часа
time = 8 * 60 # минут
max_detail = 120 # количество необходимых деталей

In [14]:
time_last = 0 # времени прошло
detail_complete = 0 # сделанных деталей
history = []
while time_last <= time:
    u = random() # случайная величина
    entrance = exp(l, u) # время поступления
    time_last += entrance # времени прошло
    
    detail = type_detail(u) # тип заготовки
    
    necessary_work = work[str(detail)] # необходимые работы
    time_to_work_mean = time_work[str(detail)] # среднее время выполнения работ
    time_to_work = [(lambda time_mean: exp(4, u))(time_mean) for time_mean in time_to_work_mean] # время на выполнение работ
    
    for id_m, closed in platform[str(detail)].items():
        if closed:
            history.append([detail, time_last, time_last + sum(time_to_work), False, id_m])
            platform[str(detail)][id_m] = False
            flag = False
            break
        flag = True
    
    if flag:
        time_min, id_plat = min_time_index_last(history, detail)
        history.append([detail, time_last, time_min, False, id_plat])
        
    for i in range(len(history)):
        if history[i][2] < time_last:
            history[i][3] = True
            platform[str(history[i][0])][str(history[i][4])] = True

In [16]:
df = pd.DataFrame([], columns=["Тип детали", "Время поступления", "Время завершения изготовления", "ID платформы", "ID станка"])

for el in history:
    if el[3]:
        df2 = pd.DataFrame([[el[0], el[1], el[2], el[0], el[4]]], columns=["Тип детали", "Время поступления", "Время завершения изготовления", "ID платформы", "ID станка"])
        df = df.append(df2, ignore_index=True)
        
df.to_excel(f"Отчет.xlsx")